<a href="https://colab.research.google.com/github/op1um/bert-nlp/blob/master/Bert_base_uncased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()
uploaded2 = files.upload()

import io
df = pd.DataFrame(pd.read_csv(io.BytesIO(uploaded["df.csv"])))

Saving df.csv to df.csv


Saving df2.csv to df2.csv


In [0]:
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

#get sentences in the right format for BERT
sentences = ["[CLS] " + s + " [SEP]" for s in df.sentence]
print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

def max_sentence(sentences):
    sentence_list = []
    for sentence in sentences:
        sentence_list.append(len(sentence))
    return max(sentence_list)

MAX_LEN = max_sentence(df.sentence) + 10

input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

# Use train_test_split to split our data into train and validation sets for training
labels = df.correct.values

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)
                                             
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. 
batch_size = 32

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
nb_labels = 2
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=nb_labels)
model.cuda()

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 92kB 6.8MB/s 
     |████████████████████████████████| 675kB 7.5MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609189 sha256=fc0cc2c8e7a3cf4eebb2899c003a5c08ff884355fcabf47d924ff0cf124c8112
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built regex


Using TensorFlow backend.


[CLS] "I sting a mosquito" is wrong because A human is a mammal [SEP]


100%|██████████| 231508/231508 [00:00<00:00, 415270.31B/s]


Tokenize the first sentence:
['[CLS]', '"', 'i', 'sting', 'a', 'mosquito', '"', 'is', 'wrong', 'because', 'a', 'human', 'is', 'a', 'mammal', '[SEP]']


100%|██████████| 407873900/407873900 [00:32<00:00, 12499633.80B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
   

In [0]:
# BERT fine-tuning parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01}, # originally 0.01
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=1e-6,
                     warmup=.1)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
  
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs 
epochs = 4

# BERT training loop
for _ in trange(epochs, desc="Epoch"):  
  
  ## TRAINING
  
  # Set our model to training mode
  model.train()  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
  ## VALIDATION

  # Put model in evaluation mode
  model.eval()
  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids) #flat_accuracy(predictions, labels)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

t_total value of -1 results in schedule not being applied
Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.5510715093877581


Epoch:  25%|██▌       | 1/4 [10:33<31:39, 633.10s/it]

Validation Accuracy: 0.7583333333333333
Train loss: 0.4704102571805318


Epoch:  50%|█████     | 2/4 [21:05<21:05, 632.95s/it]

Validation Accuracy: 0.7854166666666667
Train loss: 0.43774929108443084


Epoch:  75%|███████▌  | 3/4 [31:39<10:33, 633.08s/it]

Validation Accuracy: 0.8029166666666666
Train loss: 0.41462124899581626


Epoch: 100%|██████████| 4/4 [42:11<00:00, 632.99s/it]

Validation Accuracy: 0.8016666666666666


In [0]:
#prepare test data
test_df = pd.DataFrame(pd.read_csv(io.BytesIO(uploaded2["df2.csv"])))

#get sentences in the right format for BERT
sentences_test = ["[CLS] " + s + " [SEP]" for s in test_df.sentence]
print(sentences_test[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences_test]
print ("Tokenize the first sentence:")
print (tokenized_texts_test[0])

input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids_test = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks_test = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids_test:
  seq_mask_test = [float(i>0) for i in seq]
  attention_masks_test.append(seq_mask_test)

# test inputs and test masks
test_inputs = torch.tensor(input_ids_test)
test_masks = torch.tensor(attention_masks_test)

# prepare for predictions
batch_size = 32
prediction_data = TensorDataset(test_inputs, test_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

## Prediction on test set
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions= []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # Store predictions and true labels
  predictions.append(logits)
  
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
#flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

print(flat_predictions)

[CLS] "He poured orange juice on his cereal." is against common sense because Orange juice is usually bright orange. [SEP]
Tokenize the first sentence:
['[CLS]', '"', 'he', 'poured', 'orange', 'juice', 'on', 'his', 'cereal', '.', '"', 'is', 'against', 'common', 'sense', 'because', 'orange', 'juice', 'is', 'usually', 'bright', 'orange', '.', '[SEP]']
[array([ 1.1489844, -1.205121 ], dtype=float32), array([ 1.4811416, -1.7109603], dtype=float32), array([-0.4557183,  0.6492246], dtype=float32), array([0.2148941 , 0.05263064], dtype=float32), array([ 0.8361709, -0.7558469], dtype=float32), array([ 1.1057609, -1.0233164], dtype=float32), array([ 1.3024713, -1.4557383], dtype=float32), array([-0.7130843,  1.1039435], dtype=float32), array([ 1.0185981, -1.0706326], dtype=float32), array([ 0.5338934, -0.2916662], dtype=float32), array([ 1.107887 , -1.2640152], dtype=float32), array([-0.4534766,  0.4204891], dtype=float32), array([ 1.1082548, -0.8568938], dtype=float32), array([ 0.33560312, -0.

In [0]:
# put labels to predictions
test_df['predictions'] = flat_predictions

def pick_label(p1, p2, p3):
  pred_true = [p1[1], p2[1], p3[1]]
  max_pred = max(pred_true)
  if max_pred == p1[0]:
    return A
  if max_pred == p2[1]:
    return B
  else:
    return C

pred_correct = []
for pred in flat_predictions:
  pred_correct.append(pred[1])
test_df['pred 1'] = pred_correct

#group by id
#take max pred 1 value
#return whether that is A B or C
grouped = test_df.groupby('id')
grouped.get_group('sentence_7') 

,Unnamed: 0,sentence,ABC,id,predictions,pred 1
6,6,"""Lisa played jazz music for Moses of the Old T...",A,sentence_7,"[1.3024713, -1.4557383]",-1.455738
2006,2006,"""Lisa played jazz music for Moses of the Old T...",B,sentence_7,"[-0.15843634, 0.1649694]",0.164969
4006,4006,"""Lisa played jazz music for Moses of the Old T...",C,sentence_7,"[0.35180223, -0.18496959]",-0.184970


In [0]:
# create final csv output
final_df = test_df.loc[test_df.reset_index().groupby(['id'], sort=False)['pred 1'].idxmax()]
final_df_csv = final_df[['id', 'ABC']].set_index('id')

final_df_csv = final_df_csv.rename(columns={'ABC': 'answer'})
final_df_csv.to_csv("bert_a_4epochs_weightdecay0.01_learningrate_1e-6.csv")
files.download('bert_a_4epochs_weightdecay0.01_learningrate_1e-6.csv')

KeyboardInterrupt: ignored

In [0]:
# plot training performance just in case it shows anything interesting I dunno
T = train_loss_set

plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(T[0::400],'o-')
plt.show()